In [1]:
%load_ext sparksql_magic

In [2]:
%%sparksql
show databases

22/03/17 19:45:50 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 1.2.0
22/03/17 19:45:50 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException


databaseName
default


# Input

In [3]:
df_rent_input = spark.read.csv('./data/input/rent', header=True)

In [4]:
df_rent_input.count()

24000

In [5]:
df_rent_input.createOrReplaceTempView('rent_input')

In [6]:
%%sparksql
select distinct offset from rent_input

22/03/17 19:46:05 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


offset
1
3
5
4
2


# Output

In [7]:
df_rent_output = spark.read.parquet('./data/output/rent')

In [8]:
df_rent_output.count()

24000

In [9]:
df_rent_output.count()
df_rent_output.createOrReplaceTempView('rent_output')

In [10]:
%%sparksql
select distinct offset from rent_output

offset
1
3
5
4
2


# Reverse User_id Domain

In [13]:
df_reverse_user_id = spark.read.parquet('./data/reverse/user_id_sha512sha256')

In [14]:
df_reverse_user_id.count()

1000

In [15]:
df_reverse_user_id.createOrReplaceTempView('reverse_user_id')

# Reverse Cretit_card domain

In [16]:
df_reverse_credit_card = spark.read.parquet('./data/reverse/credit_card_sha512sha256')

In [17]:
df_reverse_credit_card.count()

24000

In [18]:
df_reverse_credit_card.createOrReplaceTempView('reverse_credit_card')

# Minus clause to validate table

In [21]:
%%sparksql 
select 
    id, credit_card, date_rent, time_rent, movie, movie_genres, offset 
from rent_input
minus 
select 
    rui.user_id as id, rcc.credit_card, date_rent, time_rent, movie, movie_genres, offset 
from rent_output as ro
INNER JOIN reverse_user_id as rui ON (ro.id = rui.user_id_mask) 
INNER JOIN reverse_credit_card as rcc ON (ro.credit_card = rcc.credit_card_mask)

id,credit_card,date_rent,time_rent,movie,movie_genres,offset


In [22]:
%%sparksql 
select
*
from rent_output

only showing top 20 row(s)


date_rent,time_rent,movie,movie_genres,id,credit_card,offset
2020-04-12,19:24:10,Blinkity Blank,Animation,abae1023971539d380f71298401acd4d48791a92b19e67d00fd452b686401e54,18e61c9b4b33d0e198bb5d91b2c451ed341d4d3a214f1f51c2cc7f98ee6547d8,1
2020-04-12,3:54:29,What Have I Done to Deserve This? (¿Qué he hecho yo para merecer esto!!),Comedy|Drama,b3fd63acc23e666fc95b24b102ab1d7ae39ec26c4c45205b2d4c027c9c9cb966,e2bea2000bbbe37c77e1773573b13d36a9057a19586a4902fe635719bea21ab2,1
2020-04-12,1:33:37,Second in Command,Action|Thriller,c6546b74e8d033688b8800d673090d264e54b3bd921ef463c063807335b4b4f2,75f77c2b11155463513424b0d56f6209923711d6388f70c65308fc24090890b6,1
2020-04-12,4:32:28,Buddha Collapsed Out of Shame,Drama|War,0747df874d1cad8841a36e09fd6933f617d11179638517e3de858e534e14d554,61c425599469eac91e185721412aacd3c33d6de97824bb6086ef5dc89b712713,1
2020-04-12,16:06:10,"Mechanic, The",Action|Drama|Thriller,6d0347ffb6b203d0bdcd592c40090c1afcaa34e2c470fb81d391435101952ee4,09aa5109a2327f531c860dca9fb7355b6546536c134093e9d7d7318719e13958,1
2020-04-12,2:03:09,"Prince & Me II: The Royal Wedding, The",Comedy|Romance,f44e68de464337affafae6e374941bfb93f31b497dc80cadfdb3fe049d372072,1fcbcd98bdb399d2bbb4e54dd5c569975dda151e313e4d250978564f778e0333,1
2020-04-12,7:23:01,Neverland,Adventure|Fantasy,7dd3f1ce8e9902f2e25ab006a6b043b1fe6a21a8ed23fb68f51164f98cd29d70,ed986ee20009536b41ec2b9d5c1b4a67355c0435f8f162b87c592d74dc90eb23,1
2020-04-12,6:39:22,Nine Dead,Crime|Mystery|Thriller,c6d180917fb29aa932990e4f91b7ba500022290113fc32fdeb7890518a0c2179,7907e94aaee1e669d0691fb249ec3a9aa593f2a0eb144bdc8427f3604d6d25cb,1
2020-04-12,19:40:11,One Magic Christmas,Drama|Fantasy,fc00e9897992742c919a403f3584958655310c45e11e8e76b30e44588be696a1,2b0c7156801270b5c5a4b21dedaa11ac8a6b320e2bf00ed0e60d7c68da570c65,1
2020-04-12,19:26:41,Sachs' Disease (La maladie de Sachs),Drama,4d1931a512802ccd241903e6143578ed79eca7e071092abd7f097ec32a514ec0,dddd2b1dc80a9ac760e762cae6a7522b53061fdb3df7413862745e8a6eaa1c57,1


In [23]:
%%sparksql 
SELECT *
    FROM rent_input

only showing top 20 row(s)


id,credit_card,date_rent,time_rent,movie,movie_genres,offset
1,4903136325452740870,2020-04-12,10:52:53,Dry Cleaning (Nettoyage à sec),Drama,1
2,3577491551960733,2020-04-12,4:42:13,Alone (Issiz adam),Drama|Romance,1
3,3536874685193000,2020-04-12,16:53:05,"Family Stone, The",Comedy|Drama|Romance,1
4,344681715118362,2020-04-12,22:55:42,"Man Who Captured Eichmann, The",Drama|War,1
5,5048375443966618,2020-04-12,5:27:32,Desperately Seeking Susan,Comedy|Drama|Romance,1
6,201621133610618,2020-04-12,15:12:07,Caine (Shark!),Action|Adventure|Thriller,1
7,4017952805033425,2020-04-12,1:45:48,The Scapegoat,Drama,1
8,4017957934466107,2020-04-12,12:35:09,"Injury to One, An",Documentary,1
9,337941060394011,2020-04-12,5:19:47,Skeleton Crew,Horror,1
10,6399834887430831,2020-04-12,1:01:10,Age of Ice,Adventure|Sci-Fi,1


In [20]:
%%sparksql 
SELECT * from rent_output

only showing top 20 row(s)


date_rent,time_rent,movie,movie_genres,id,credit_card,offset
2020-04-12,21:48:57,Himizu,Children|Crime|Drama,5a3d36f3c5fd7c57f33c564590ff5132a999c9c52934f7d2d76773231a14ee1a,24ce6a634ce326653cc3369ffaba5077dde7029f4d5dfd3b5500b757909801fd,1
2020-04-12,7:23:01,Neverland,Adventure|Fantasy,7dd3f1ce8e9902f2e25ab006a6b043b1fe6a21a8ed23fb68f51164f98cd29d70,ed986ee20009536b41ec2b9d5c1b4a67355c0435f8f162b87c592d74dc90eb23,1
2020-04-12,4:26:29,99 francs,Comedy,e75d986b87db8a9f803ecf8388e8d2f60817c5cf14dd32d566f01a161a08ea3f,186c0ba3bc0e0d47892882b5fa298a24b6be89ed0c0541d3d485bb9f6b7cdf97,1
2020-04-12,19:26:41,Sachs' Disease (La maladie de Sachs),Drama,4d1931a512802ccd241903e6143578ed79eca7e071092abd7f097ec32a514ec0,dddd2b1dc80a9ac760e762cae6a7522b53061fdb3df7413862745e8a6eaa1c57,1
2020-04-12,13:13:24,Cabin Boy,Comedy,99fc323d8ed63269908df021afd3f5e71a945fdf5ae76e081d7d6eebb56cb8a2,e9d0136c647b810cfd4837add2fb31a5072ab1074da305eaffa1f742001bd13a,1
2020-04-12,1:20:44,Laila,Drama|Romance,a992199c0b5a96c67ef35d54367bb373d86d025607948d07f87fefe3ed896fd0,5152d7d99e156aafb8f527cbe0f5a8ecb2f888c68dfedeca5f5ac20b22e2779a,1
2020-04-12,14:45:48,"Magnificent Seven, The",Adventure|Western,cc836f5987b01318435a75ff6d0e6598b8902f2cc9e14e9d6d787edba366deb9,a0081d6c93bf48fb9577513d326be5f226e0bd20e3853045cfaf3ea9dd3fbb35,1
2020-04-12,6:24:04,Beautiful,Drama|Thriller,666a5391d3a9e296dae35207d4fba459c33bb64dcc3e56019699b3b51f1d289f,ca6aebaef15fd1e193767adfff913b072c17787d6a91a835f5e1cc625f843a92,1
2020-04-12,13:53:53,U Turn,Crime|Drama|Mystery,abeab0626396b2a733f517eae43287e38ced15ce7006296a385e2f9a56ea4517,d107052dc40b767b532d327020e03b3d664db3ecdf0bbc6bac3908316fc06e75,1
2020-04-12,1:17:46,Kill the Irishman,Action|Crime,29c6ebf11da7419171f8f780dfffa6f92aed43a81c553c6cebd3e2f9aa3e91ec,0415c7791fcea322c8ab2cc8663364e24723151bcb749159430cc763970d628d,1
